This is the main notebook of the report, it'll contain all of our findings and developments.

In [1]:
import DataReader

x_test, x_train, y_test, y_train = DataReader.split_sheet_into_test_training_per_word()
